This script will scan any folder you point it to, find Python functions with docstrings, and create a training pair for each one.

### The Code Parser
We use Python's built-in ast (Abstract Syntax Tree) module. This is much safer and more accurate than using Regex to find code.

In [1]:
import ast
import os
import json
import pandas as pd

SOURCE_FOLDER = "/content/Python codes"
OUTPUT_FILE = "docstring_data.json"

def remove_docstring(node):
    """
    Takes an AST Function node and removes the docstring.
    Returns the source code of the function without the docstring.
    """
    if not (node.body and isinstance(node.body[0], ast.Expr) and isinstance(node.body[0].value, ast.Constant)): return None
    docstring = node.body[0].value.value
    return docstring

def extract_function_pairs(file_path):
    pairs = []
    try:
        with open(file_path, "r", encoding="utf-8") as f: source = f.read()
        tree = ast.parse(source)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                docstring = ast.get_docstring(node)
                if docstring:
                    start_lineno = node.lineno - 1
                    end_lineno = node.end_lineno

                    lines = source.splitlines()
                    full_function_code = "\n".join(lines[start_lineno:end_lineno])

                    input_code = full_function_code.replace(f'"""{docstring}"""', '')
                    input_code = input_code.replace(f"'''{docstring}'''", '')

                    if len(input_code) > 50:
                        pairs.append({
                            "input_code": input_code.strip(),
                            "target_code": full_function_code.strip()
                        })

    except Exception as e: pass
    return pairs

### Installing new libs for best working:

In [4]:
!mkdir -p /content/training_source
!pip install requests flask numpy -t /content/training_source

print("✅ Downloaded libraries! We now have thousands of professional functions to scrape.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 103.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 19.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

✅ Downloaded libraries! We now have thousands of professional functions to scrape.


### The Scraper:

In [1]:
import ast
import os
import json

SOURCE_FOLDER = "/content/training_source"
OUTPUT_FILE = "docstring_data.json"

def extract_function_pairs(file_path):
    pairs = []
    try:
        with open(file_path, "r", encoding="utf-8") as f: source = f.read()

        tree = ast.parse(source)
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                docstring = ast.get_docstring(node)
                if docstring:
                    start = node.lineno - 1
                    end = node.end_lineno
                    lines = source.splitlines()
                    full_code = "\n".join(lines[start:end])

                    input_code = full_code.replace(f'"""{docstring}"""', '')
                    input_code = input_code.replace(f"'''{docstring}'''", '')

                    if len(input_code) > 40:
                        pairs.append({
                            "input_code": input_code.strip(),
                            "target_code": full_code.strip()
                        })
    except:
        pass
    return pairs

# --- MAIN LOOP ---
all_data = []
print(f"🕵️ Scanning {SOURCE_FOLDER}...")

for root, dirs, files in os.walk(SOURCE_FOLDER):
    for file in files:
        if file.endswith(".py"):
            pairs = extract_function_pairs(os.path.join(root, file))
            all_data.extend(pairs)

print(f"✅ Found {len(all_data)} training examples!")

# Save
with open(OUTPUT_FILE, "w") as f:
    json.dump(all_data, f, indent=4)

🕵️ Scanning /content/training_source...
✅ Found 3432 training examples!


### Just checking what is there to look out:

In [2]:
ls "/content/training_source"

bin/                                 itsdangerous-2.2.0.dist-info/
blinker/                             jinja2/
blinker-1.9.0.dist-info/             jinja2-3.1.6.dist-info/
certifi/                             markupsafe/
certifi-2025.11.12.dist-info/        markupsafe-3.0.3.dist-info/
charset_normalizer/                  numpy/
charset_normalizer-3.4.4.dist-info/  numpy-2.3.5.dist-info/
click/                               numpy.libs/
click-8.3.1.dist-info/               requests/
flask/                               requests-2.32.5.dist-info/
flask-3.1.2.dist-info/               urllib3/
idna/                                urllib3-2.6.1.dist-info/
idna-3.11.dist-info/                 werkzeug/
itsdangerous/                        werkzeug-3.1.4.dist-info/
